In [1]:
import numpy as np 
import pandas as pd 

"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

# Info
 80 variable columns 

In [2]:
#Read data
train_set = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_set = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

#ID column
Id_column = test_set['Id'].copy()
#Id_column

In [3]:

#Distincting between train set and test test to merge
train_set['train'] = 1
test_set['train'] = 0


In [4]:
#Merging train and test set for data cleaning

all_data = pd.concat( [train_set,test_set])
#all_data

In [5]:

#Dropping ID and SalePrice off the all_data to clean it then re-split
all_data.drop(columns=['Id','SalePrice'],inplace=True)                       
#all_data.head()


In [6]:
# Finding all categorical features #2
categ_features=all_data.select_dtypes(include=['object']).columns
#categ_features


In [7]:
#Filling missing values for int variables based on looks
[all_data.loc[:,col].fillna(all_data[col].mean(),inplace=True) for col in all_data.columns if col not in categ_features]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
#Filling missing values for categorical variables without ignoring fatal stuff
[all_data.loc[:,col].fillna('_Non_',inplace=True) for col in all_data.columns if col in categ_features]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [9]:
#Getting dummy variables and transforming into onehot
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#Finding columns indexes
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]
#ct = ColumnTransformer([('encoder',OneHotEncoder(),column_index(all_data,categ_features))],remainder='passthrough')
#onehotencoder = OneHotEncoder(categories = list(column_index(all_data,categ_features)))
#all_data = onehotencoder.fit_transform(all_data)

all_data = pd.get_dummies(all_data,columns=categ_features,drop_first=True)

In [10]:
# Seperate into test and train sets

x_train = all_data.loc[all_data['train']==1].copy() 
y_train = train_set['SalePrice'].copy()

x_test = all_data.loc[all_data['train']==0].copy()

In [11]:
#Using all-time favourite X-treme Gradient Boost
from xgboost import XGBRegressor

regressor = XGBRegressor() 
regressor.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
from catboost import CatBoostRegressor

regressor2 = CatBoostRegressor() 
regressor2.fit(x_train, y_train)

Learning rate set to 0.04175
0:	learn: 77334.8360834	total: 57.4ms	remaining: 57.3s
1:	learn: 75363.1415734	total: 63.7ms	remaining: 31.8s
2:	learn: 73324.4092192	total: 68.9ms	remaining: 22.9s
3:	learn: 71463.1890859	total: 74.2ms	remaining: 18.5s
4:	learn: 69703.2763613	total: 79.5ms	remaining: 15.8s
5:	learn: 68109.5788659	total: 85.2ms	remaining: 14.1s
6:	learn: 66386.9466531	total: 91.5ms	remaining: 13s
7:	learn: 64810.8139497	total: 96.9ms	remaining: 12s
8:	learn: 63397.1941988	total: 102ms	remaining: 11.2s
9:	learn: 61801.8863200	total: 107ms	remaining: 10.6s
10:	learn: 60385.8407972	total: 114ms	remaining: 10.2s
11:	learn: 58986.4077304	total: 122ms	remaining: 10s
12:	learn: 57654.1563285	total: 128ms	remaining: 9.7s
13:	learn: 56404.5753545	total: 136ms	remaining: 9.55s
14:	learn: 55119.2247856	total: 141ms	remaining: 9.27s
15:	learn: 53973.0239783	total: 148ms	remaining: 9.13s
16:	learn: 52874.8301694	total: 155ms	remaining: 8.94s
17:	learn: 51790.5728701	total: 160ms	remaini

In [13]:
y_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')['SalePrice']

In [14]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = x_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 85.50 %
Standard Deviation: 5.21 %


In [15]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor2, X = x_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Learning rate set to 0.040974
0:	learn: 78447.1729897	total: 5.66ms	remaining: 5.65s
1:	learn: 76501.1465100	total: 10.3ms	remaining: 5.13s
2:	learn: 74682.4695043	total: 15ms	remaining: 4.97s
3:	learn: 72820.1397536	total: 19.9ms	remaining: 4.95s
4:	learn: 71104.1296463	total: 25ms	remaining: 4.97s
5:	learn: 69471.8953814	total: 30.2ms	remaining: 5s
6:	learn: 67829.8540656	total: 35.7ms	remaining: 5.06s
7:	learn: 66139.6094407	total: 40.7ms	remaining: 5.04s
8:	learn: 64758.5633758	total: 45.6ms	remaining: 5.02s
9:	learn: 63431.2687455	total: 50.5ms	remaining: 5s
10:	learn: 62021.7594232	total: 56.1ms	remaining: 5.04s
11:	learn: 60619.6185035	total: 61.3ms	remaining: 5.05s
12:	learn: 59268.8228732	total: 66.3ms	remaining: 5.04s
13:	learn: 58127.8066600	total: 71.8ms	remaining: 5.06s
14:	learn: 56920.7144813	total: 76.7ms	remaining: 5.04s
15:	learn: 55634.6426736	total: 81.3ms	remaining: 5s
16:	learn: 54480.5878004	total: 87ms	remaining: 5.03s
17:	learn: 53406.7975166	total: 92.2ms	rema

In [16]:
#Submition File
y_pred = regressor2.predict(x_test)
y_pred

pred = pd.DataFrame(y_pred)
pred = pd.concat([Id_column,pred],axis=1)
pred.columns = ['Id','SalePrice']



In [17]:
pred.to_csv('predictions.csv',index=False)